<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C6-Extending_Burp_Proxy/Turning_Website_Content_Into_Password_Gold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Turning Website Content into Password Gold__*

Many times, security comes down to one thing: User passwords. It's sad but true. Making things worse, when it comes to web applications, especially custom ones, it's all too common to discover that they don't lock users out of their accounts after a certain number of failed authentication attempts. In other instances, they don't enforce strong passwords. In these cases, an online password guessing session like the one in the last chapter might be just the ticket to gain access to the site.

The trick to online password guessing is getting the right word list. You can't test 10 million passwords if you're in a hurry, so you need to be able to create a word list targeted to the site in question. Of course, there are scripts in Kali Linux that crawl a website and generate a word list based on site content. But if you've already used Burp to scan the site, why send more traffic just to generate a word list? Plus, those scripts usually have a ton of command line arguments to remember. If you're anything like us, you've already memorized enough command line arguments to impress your friends, so let's make Burp do the heavy lifting.

Open __bhp_wordlist.py__ and knock out this code:

In [ ]:
from burp import IBurpExtender
from burp import IContextMenuFactory

from java.util import ArrayList
from java.swing import JMenuItem

from datetime import datetime
from HTMLParser import HTMLParser

import re

class TagStripper(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.page_text = []

    def handle_data(self, data):
        self.page_text.append(data) #[1]

    def handle_comment(self, data):
        self.page_text.append(data) #[2]

    def strip(self, html):
        self.feed(html)
        return " ".join(self.page_text) #[3]

class BurpExtender(IBurpExtender, IContextMenuFactory):
    def registerExtenderCallbacks(self, callbacks):
        self._callbacks = callbacks
        self._helpers = callbacks.getHelpers()
        self.context = None
        self.hosts = set()

        # Start with something we know is common
        self.wordlist = set(["password"]) #[4]

        # We set up our extension
        callbacks.setExtensionName("BHP Wordlist")
        callbacks.registerContextMenuFactory(self)

        return

    def createMenuItems(self, context_menu):
        self.context = context_menu
        menu_list = ArrayList()
        menu_list.add(JMenuItem("Create Wordlist", actionPerformed=self.wordlist_menu))

        return menu_list

The code in this listing should be pretty familiar by now. We start by importing the required modules. A helper __TagStripper__ class will allow us to strip the HTML tags out of the HTTP responses we process later on. Its __handle_data__ method store the page text __[1]__ in a member variable. We also define the __handle_comment__ method because we want to add the words stored in developer comments to the password list as well. Under the covers, __handle_comment__ just calls __handle_data__ __[2]__ (in case we want to change how we process page text down the road).

The __strip__ method feeds HTML code to the base class, __HTMLParser__, and returns the resulting page text __[3]__, which will come in handy later. The rest is almost exactly the same as the start of the __bhp_bing.py__ script we just finished. Once again, the goal is to create a context menu item in the Burp UI. The only thing new here is that we store our word list in a __set__, which ensures that we don't introduce duplicate words as we go. We initialize the __set__ with everyone's favorite password, __password__ __[4]__, just to make sure it ends up in our final list.

Now let's add the logic to take the selected HTTP traffic from Burp and turn it into a base word list: